In [60]:
import json
import string
import random 
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
# nltk.download("punkt")
# nltk.download("wordnet")
# nltk.download('omw-1.4')

In [61]:
f=open('newdata.json')
data=json.load(f)

In [62]:
data.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'])

In [63]:
f=0
for intents in data:
  print(f)
  #print(data['0']['Patterns'])
  f+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [64]:

#nltk.download('stopwords')
#nltk.download('wordnet')


import string
def text_process(text):

    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

In [65]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords.words('english').append(x for x in ['dear','student','students','Please','please','sir','dear'])

In [66]:
lm = WordNetLemmatizer()
ourClasses = []
newWords = []
documentX = []
documentY = []
for intent in data.keys():
    for pattern in data[intent]['Patterns']:
        #print(pattern)
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent)

    if intent not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent)
newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation if word not in stopwords.words('english')] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [67]:
trainingData = []
s="" 
outEmpty = [0] * len(ourClasses)
for idx, doc in enumerate(documentX):
    bagOfwords = []
    s=""
    k=doc.split()
    for x in k:
        x=x.lower()
        if x not in string.punctuation and x not in stopwords.words('english'):
            s+=x+" "
    text = lm.lemmatize(s)
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

In [68]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [69]:
iShape = (len(x[0]),)
oShape = len(y[0])

ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))

ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "sigmoid"))
ourNewModel.add(Dense(oShape,activation="softmax"))
md = tensorF.keras.optimizers.Adam()
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=400, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               212864    
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 15)                975       
                                                                 
 dense_11 (Dense)            (None, 15)                240       
                                                                 
Total params: 222,335
Trainable params: 222,335
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400
19/19 [========================

In [70]:
from sklearn.metrics import multilabel_confusion_matrix,ConfusionMatrixDisplay
y_test=ourNewModel.predict(x)
#multilabel_confusion_matrix(y,y_test)
y_test

19/19 [==============================] - 0s 2ms/step


array([[9.94197011e-01, 1.44640726e-04, 1.96354977e-05, ...,
        7.01866375e-05, 1.00762714e-04, 3.21671882e-06],
       [2.37512486e-05, 3.94367671e-04, 1.38456264e-04, ...,
        9.05722914e-07, 5.32995800e-05, 8.03739204e-06],
       [6.28620153e-04, 1.59481657e-04, 7.48810417e-05, ...,
        1.18421239e-03, 1.05425622e-03, 4.37512877e-04],
       ...,
       [2.56189942e-05, 4.12601657e-04, 1.37289215e-04, ...,
        9.17840794e-07, 5.41412555e-05, 8.17992350e-06],
       [2.77462514e-05, 9.98958945e-01, 4.85386984e-08, ...,
        7.89178785e-07, 4.60379823e-08, 3.77683791e-05],
       [1.09746610e-03, 3.38467042e-04, 1.07947295e-03, ...,
        1.63436204e-03, 9.67699349e-01, 2.02912278e-03]], dtype=float32)

In [71]:
l=[x for x in data.keys()]
l

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14']

In [72]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word.lower()) for word in newtkns if word not in string.punctuation if word not in stopwords.words('english')]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  print(yp)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, data):
  brr=" "
  tag = firstlist[0]
  listOfIntents = [x for x in data.keys()]
  for i in listOfIntents:
    if i == tag:
      #print("waaw")    ce(data[i]['Responses'])
      break
  return brr

In [87]:
i=0
while i<3:
    newMessage = input()
    intents = Pclass(newMessage, newWords, ourClasses)
    print(getRes(intents, data))
    i+=1

1/1 [==============================] - 0s 27ms/step
[[5, 0.57338345]]
Dear Student,

Your request has been received and your course will be updated shortly.
Thanks.
1/1 [==============================] - 0s 25ms/step
[[1, 0.990099]]
Dear student, your request has been updated on collpoll, kindly check
1/1 [==============================] - 0s 24ms/step
[[6, 0.44913656], [4, 0.41162652]]
Dear Varnika, Please contact to  me on my phone number 7503699484/8178021662 . Whenever you are free  for this regards


In [74]:
testi=[]
for i in data.keys():
    j=data[i]['Patterns']
    k=data[i]['Responses']
    testi.append([j,k])

In [75]:
testi

[[['i am a 4th year(7th sem) btech CSE student and I want to drop application of AI course which is an open elective as I will be doing an internship for 6 months',
   'need batch changed from b1 to b2. conflict of interest with certain individuals, best way to avoid more drama is to seperate us.',
   "I wanted to change my batch from EB10 to EB02 but the option is mot available. My all the teammates for Capstone project is in EB02 . It's becoming difficult for me to manage as Capstone is the biggest project of the complete course.",
   'Sir, In batch change option I wanted to change my batch to EB-25 which was reflecting there but when I clicked on it. The page crashed and when I reload it, it was showing that my batch is changed to EB-21 but I wanted it to change to EB-25.',
   "Sir, I wanted to change my batch to EB-25 but due to some error it is changed to EB-21 sir please do change it to EB-25 coz I don't want to lose this opportunity. coz this is the last time we can change batch

In [76]:
import numpy as np
testi=np.array(testi)

C:\Users\devlo\AppData\Local\Temp\ipykernel_3384\2290509653.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  testi=np.array(testi)


In [86]:
j=0
wrng=[]
corec=[]
x=1
for i in testi:
    for k in i[0]:
            newMessage = k
            intents = Pclass(newMessage, newWords, ourClasses)
            f=getRes(intents, data)
            if f not in i[1]:
                if(x==7):
                        x+=1
                p="Category "+str(x)
                wrng.append([k,f,p])
                x+=1
            else:
                 j+=1
    corec.append([f,f+j])
    f=0
    j=0

1/1 [==============================] - 0s 25ms/step
[[0, 0.96631527]]
1/1 [==============================] - 0s 91ms/step
[[0, 0.9975479]]
1/1 [==============================] - 0s 72ms/step
[[0, 0.9892658]]
1/1 [==============================] - 0s 64ms/step
[[0, 0.9946906]]
1/1 [==============================] - 0s 57ms/step
[[6, 0.516933], [0, 0.4767388]]
1/1 [==============================] - 0s 54ms/step
[[0, 0.5871327], [7, 0.33325127]]
1/1 [==============================] - 0s 36ms/step
[[0, 0.9975376]]
1/1 [==============================] - 0s 33ms/step
[[0, 0.9971113]]
1/1 [==============================] - 0s 39ms/step
[[0, 0.99680233]]
1/1 [==============================] - 0s 43ms/step
[[0, 0.9969464]]
1/1 [==============================] - 0s 53ms/step
[[0, 0.9969506]]
1/1 [==============================] - 0s 69ms/step
[[0, 0.9975358]]
1/1 [==============================] - 0s 65ms/step
[[0, 0.86668056]]
1/1 [==============================] - 0s 29ms/step
[[6, 0.6941138],

TypeError: can only concatenate str (not "int") to str

In [78]:
from sklearn.metrics import r2_score
r2_score(y_test,y)

0.8872657358120635

In [79]:
import pandas as pd
wrng=pd.DataFrame(wrng,columns=["Input","Output Displayed","Category"])

In [80]:
wrng.to_csv('wrng.csv')

In [81]:
for x,i in enumerate(corec):
    print("Category {}".format(x),i[0]/i[1]*100)

In [82]:
wrng

,Input,Output Displayed,Category
0,"Sir, I wanted to change my batch to EB-25 but ...","probably yes, and also you will get an intimat...",Category 1
1,Ok sir,"Dear Sumit, Sorry to hear that dear, you can g...",Category 2
2,i want to drop my course of entrepreneurship a...,"Dear Student, I m closing this quary as this ...",Category 3
3,Need results for all semesters,"Dear Student,\nit is done, kindly refresh and ...",Category 4
4,I am not able to upload the inital goal report,Dear Student kindly refer to the mail shared b...,Category 5
5,I want to change my batch,"Dear Student, It will be visible on LMS this w...",Category 6
6,\nI have opted for internship elective this se...,"Dear Student, there was no error in the system...",Category 8
7,This is what shown in app,"Dear Student, you will see the option to the t...",Category 9
8,Any update?,"Dear Student, Kindly share the query.",Category 10
9,And I know sir you are the only one who can do...,"Dear Student, Specialization change is not pos...",Category 11


In [83]:
data['2']

{'Patterns': ['Please sir do it',
  'Sir i  my course registration  it showing gaming specialization but i have done registration  of only cybersecurity so pleasehelo',
  'change open elective ',
  "Respected sir/mam,\n\nI have requested for batch change it showed approved in the app but not yet updated in collpoll and I haven't received approved mail to  my outlook please help me out\n\nThank you."],
 'Responses': ['Dear Student, there was no error in the system, now changing of batch is not possible',
  'Dear Student, you need to raise your request through Add/drop feature on CollPoll for dropping the course',
  'Dear Student,\n\nyou need to fill the add/drop form.\nit is shared in the mail by registrar, it is there in the registration guidelines in annexure 3\nThanks',
  'Dear Student,\n\nThe batch change window was open from 3 PM to 6 PM on 16/01/23\nNo request of batch change is possible now.\nyou can try in next semester.\nThanks.']}

In [84]:
data.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'])